# 文件加载、存储与文件格式

## 1.读写文本格式的数据

In [10]:
import numpy as np
import pandas as pd 
from pandas import Series,DataFrame

In [16]:
!type examples\ex1.csv 

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [18]:
df = pd.read_csv('examples\ex1.csv')
df   #以逗号分隔，使用read_csv将其读入一个DataFrame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [129]:
pd.read_table('examples\ex1.csv',sep=',')  #用read_table 并制定分隔符

D:\Anaconda111\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead.
  """Entry point for launching an IPython kernel.


,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


并不是所有文件都有标题行：

In [21]:
!type examples\ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [23]:
pd.read_csv('examples\ex2.csv',header=None)   #读入该文件可以让pandas为其分配默认的列名

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [25]:
pd.read_csv('examples\ex2.csv',names=['a','b','c','d','messags']) #也可以用names=[] 为其定义列名

,a,b,c,d,messags
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


如果希望将message列做成DataFrame的索引，可以通过index_col参数制定'message'

In [27]:
names = ['a','b','c','d','message']
pd.read_csv('examples\ex2.csv',names=names,index_col='message') #index_col  索引列

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


将多个列做成一个层次化索引，只需传入由列编号或列名组成的列表即可：

In [28]:
!type examples\csv_mindex.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In [29]:
parsed=pd.read_csv('examples\csv_mindex.csv',index_col = ['key1','key2'])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In [30]:
list(open('examples\ex3.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

In [31]:
result = pd.read_table('examples\ex3.txt',sep='\s+')
result

D:\Anaconda111\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead.
  """Entry point for launching an IPython kernel.


,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


In [32]:
!type examples\ex4.csv

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [33]:
pd.read_csv('examples\ex4.csv',skiprows=[0,2,3]) #用skiprows=[] 来跳过文件的哪些行

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


缺失值的处理

In [34]:
!type examples\ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo


In [36]:
result = pd.read_csv('examples\ex5.csv')
result         #pandas会用常见的标记值进行识别缺失的数据，如NaN，null

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [37]:
pd.isnull(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


na_values可以用一个列表或集合的字符串表示缺失值： 将数组中的数据改成NaN！！

In [49]:
result = pd.read_csv('examples\ex5.csv',na_values=['null'])
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [48]:
result = pd.read_csv('examples\ex5.csv',na_values='foo')
result   #将foo替换成缺失值

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


In [50]:
sentinels = {'message':['foo','NA'],'something':['two']}
pd.read_csv('examples\ex5.csv',na_values = sentinels)      #以字典的形式将数组列中的值换成NaN

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


## 逐块读取文本文件

In [51]:
pd.options.display.max_rows = 10

In [52]:
result = pd.read_csv('examples\ex6.csv')
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


In [53]:
pd.read_csv('examples\ex6.csv',nrows=5)  #只想读取几行，通过nrows= 来指定

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


要逐块读取文件，可以指定chunksize（行数）

In [56]:
chunker = pd.read_csv('examples\ex6.csv',chunksize=1000)
chunker

In [59]:
chunker = pd.read_csv('examples\ex6.csv',chunksize=1000)
tot = pd.Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(),fill_value=0)
tot = tot.sort_values(ascending = False)    
tot[:10]

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

 ### 将数据写出到文本格式 

In [61]:
data = pd.read_csv('examples\ex5.csv')
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [63]:
data.to_csv('examples\out.csv')
!type examples\out.csv        #将数据写到一个以逗号分隔的文件中  to_csv()  !type  

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


In [65]:
import sys 
data.to_csv(sys.stdout,sep='|')  #用其他分隔符 sep=

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


In [66]:
data.to_csv(sys.stdout,na_rep = 'NULL')   #缺失值在输出结果中会被表示为空字符串，可以将其表示为别的标记值 na_rep=''

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


In [67]:
data.to_csv(sys.stdout,index=False,header=False) #没有设置其他选项，则会写出行和列的标签，也可以禁用：

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


In [68]:
data.to_csv(sys.stdout,index=False,columns=['a','b','c'])

a,b,c
1,2,3.0
5,6,
9,10,11.0


Series也有一个to_csv的方法:

In [82]:
dates = pd.date_range('1/1/2000',periods=7)
ts = pd.Series(np.arange(7),index = dates)
ts.to_csv('examples/tseries.csv')
!type examples\tseries.csv

D:\Anaconda111\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  This is separate from the ipykernel package so we can avoid doing imports until


2000-01-01,0
2000-01-02,1
2000-01-03,2
2000-01-04,3
2000-01-05,4
2000-01-06,5
2000-01-07,6


### 处理分隔符格式

In [87]:
!type examples\ex7.csv 

"a","b","c"
"1","2","3"
"1","2","3"


In [107]:
import csv
f = open('examples\ex7.csv')
reader = csv.reader(f)
for line in reader:
    print(line)         #调用python内置的csv模块，csv.reader生成reader对象，对这个reader进行迭代将会为每行产生一个元祖（并移除所有的引号）

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


In [104]:
with open('examples\ex7.csv') as f:
    lines = list(csv.reader(f))
header,values=lines[0],lines[1:]   #将lines 分为标题行和数据行
data_dict = {h:v for h,v in zip(header,zip(*values))}
data_dict           #字典构造式和zip（*values），后者将行转置为列，创建数据列的字典

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

In [122]:
f = open('examples\ex7.csv')
class my_dialect(csv.Dialect):
    lineterminator = '\n'
    delimiter = ';'
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL
reader = csv.reader(f,dialect=my_dialect)   
for line  in reader:
    print(line)

['a,"b","c"']
['1,"2","3"']
['1,"2","3"']


In [119]:
f = open('examples\ex7.csv')
reader = csv.reader(f,delimiter='|')
for line  in reader:
    print(line)

['a,"b","c"']
['1,"2","3"']
['1,"2","3"']


In [126]:
with open('examples\mydata.csv','w') as f:
    writer = csv.writer(f,dialect=my_dialect)
    writer.writerow(['one','two','three'])
    writer.writerow(['1','2','3'])
    writer.writerow(['4','5','6'])
    writer.writerow(['7','8','9'])

### JSON 数据

In [6]:
obj = """
{"name":"Wes",
 "places_lives":["US","Spain","Germany"],
 "pet":null,
 "siblings":[{"name":"Scott","age":30,"pets":["Zeus","Zuko"]},
             {"name":"Katie","age":38,"pets":["Sixes","Stache","Cisco"]}]
}
"""

In [7]:
import json
result = json.loads(obj)
result                 #json.loads()可将JSON字符串转换成Python形式

{'name': 'Wes',
 'places_lives': ['US', 'Spain', 'Germany'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 30, 'pets': ['Zeus', 'Zuko']},
  {'name': 'Katie', 'age': 38, 'pets': ['Sixes', 'Stache', 'Cisco']}]}

In [9]:
asjson = json.dumps(result)
asjson

'{"name": "Wes", "places_lives": ["US", "Spain", "Germany"], "pet": null, "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]}, {"name": "Katie", "age": 38, "pets": ["Sixes", "Stache", "Cisco"]}]}'

In [15]:
siblings = pd.DataFrame(result['siblings'],columns=['name','age','pets'])
siblings  #将（一个或一组）JSON对象转化昵称DataFrame或其他便于分析的数据结构表   最简单方便的方式是
          # 向DataFrame构造器传入一个字典的列表（就是原先的JSON对象），并选取数据字段的子集

,name,age,pets
0,Scott,30,"[Zeus, Zuko]"
1,Katie,38,"[Sixes, Stache, Cisco]"


pandas.read_json可以自动将特别格式的JSON对象数据集转成为Series或DataFrame

In [16]:
!type examples\example.json

[{"a": 1, "b": 2, "c": 3},
 {"a": 4, "b": 5, "c": 6},
 {"a": 7, "b": 8, "c": 9}]


In [17]:
data = pd.read_json('examples\example.json')
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


将数据从pandas输出到JSON，可以使用to_json()

In [18]:
print(data.to_json())

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}


In [19]:
print(data.to_json(orient='records'))

[{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]


 ### XML和HTML：Web信息收集

In [26]:
tables = pd.read_html('examples\\fdic_failed_bank_list.html')
len(tables)   #read_html()可以使用lxml和BS 自动将HTML文件中的表格解析为DataFrame对象

1

In [28]:
failures = tables [0]
failures.head()                 #head() 仅查看前五行

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


In [31]:
close_timestamps = pd.to_datetime(failures['Closing Date'])
close_timestamps.dt.year.value_counts()

2010    157
2009    140
2011     92
2012     51
2008     25
2013     24
2014     18
2002     11
2015      8
2016      5
2004      4
2001      4
2007      3
2003      3
2000      2
Name: Closing Date, dtype: int64

利用lxml.objectify解析XML

In [32]:
from lxml import objectify
path = 'datasets\\mta_perf\\Performance_MNR.xml' 
parsed = objectify.parse(open(path))
root = parsed.getroot()

FileNotFoundError: [Errno 2] No such file or directory: 'datasets\\mta_perf\\Performance_MNR.xml'

## 2.二进制数据格式   pd.to_pickle方法

In [33]:
frame = pd.read_csv('examples\\ex1.csv')
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [35]:
frame.to_pickle('examples\\frame_pickle')   #pd.to_pickle存储
pd.read_pickle('examples\\frame_pickle')    #pd.read_pickle 读取

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


### pandas的两个二进制数据格式：HDF5和MessagePack 

使用HDF5格式         pd.HDFStore() 访问HDF5文件

In [40]:
frame = pd.DataFrame({'a':np.random.randn(100)})
store = pd.HDFStore('mydata.h5')
store['obj1'] = frame
store['obj1_col'] = frame['a']
store

<class 'pandas.io.pytables.HDFStore'>
File path: mydata.h5

In [41]:
store['obj1']

,a
0,-0.376244
1,-2.042679
2,1.648778
3,-0.129168
4,0.824648
5,1.447159
6,-0.043056
7,1.034068
8,-0.005415
9,-0.349832


HDFStore 支持两种存储模式：'fixed'  'table'

In [43]:
store.put('obj2',frame,format='table')
store.select('obj2',where=['index>=10 and index <=15'])

,a
10,-0.134537
11,-0.291827
12,-2.203438
13,0.514624
14,-0.560163
15,-0.428742


In [44]:
store.close()

put 是store['obj2']=frame 方法的显示版本，允许我们设置其他选项，比如格式

pandas.read_hdf 函数可以快捷使用这些工具：

In [45]:
frame.to_hdf('madata.h5','obj3',format='table')
pd.read_hdf('mydata.h5','obj3',where=['index<5'])

ValueError: The file 'mydata.h5' is already opened, but not in read-only mode (as requested).

### 读取Excel文件 

要使用ExcelFile，通过传递一个xls或xlsx创建一个实例：

In [46]:
xlsx = pd.ExcelFile('examples\\ex1.xlsx')
pd.read_excel(xlsx,'Sheet1')       #存储在表单中的数据可以用read_excel读取到DataFrame

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


In [47]:
frame = pd.read_excel('examples\\ex1.xlsx','Sheet1')
frame           #如果读取一个文件中的多个表单，创建ExcelFile会更快
                # 也可以将文件名传递到pandas.read_excel中

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


如果要将pandas数据写入到Excel格式，必须首先创建一个ExcelWriter，然后使用pandas对象的to_excel方法将数据写入其中：

In [ ]:
writer = pd.ExcelWriter('examples\\ex2.xlsx')
frame.to_excel(writer,'Sheet1')   #  将对象frame 写入到writer的表单Sheet1中
writer.save()         

In [49]:
frame.to_excel('examples\\ex2.xlsx')  # 也可以不适用ExcelWriter，而是传递文件的路径到to_excel

###  Web APIs 交互

In [50]:
import requests

In [51]:
url = 'https://api.github.com/repos/pandas-dev/pandas/issues'
resp = requests.get(url)
resp     #200 请求成功

<Response [200]>

In [52]:
data = resp.json()
data[0]['title']

'TST: test passing index (and other iterables) to .loc'

In [55]:
issue = pd.DataFrame(data,columns = ['number','title','labels','state'])
issue      #data中的每个元素都是一个包含数据得字典，我们可以直接传递数据到DataFrame中，并提取感兴趣的字段

,number,title,labels,state
0,27120,TST: test passing index (and other iterables) ...,"[{'id': 127685, 'node_id': 'MDU6TGFiZWwxMjc2OD...",open
1,27119,PERF: do not instantiate IndexEngine for stand...,[],open
2,27118,Plotting ExtensionArrays,[],open
3,27117,Make pandas.to_parquet handles partition colum...,[],open
4,27116,Bump python_requires to 3.5.2,"[{'id': 129350, 'node_id': 'MDU6TGFiZWwxMjkzNT...",open
5,27114,Decoupled more xlrd reading tests from openpyxl,"[{'id': 49254273, 'node_id': 'MDU6TGFiZWw0OTI1...",open
6,27111,TST: openpyxl tests fail if xlrd is not installed,"[{'id': 49254273, 'node_id': 'MDU6TGFiZWw0OTI1...",open
7,27110,BUG: Fix indexing on DatetimeBlock,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
8,27109,DEPR: remove deprecated date casting; closes #...,"[{'id': 87485152, 'node_id': 'MDU6TGFiZWw4NzQ4...",open
9,27108,API: Add property to Index to disallow duplicates,"[{'id': 35818298, 'node_id': 'MDU6TGFiZWwzNTgx...",open


 ## 4.数据库交互

将数据从SQL加载到DataFrame中：我们使用SQLite数据库（通过python内置的sqlite3驱动器）

In [56]:
 import sqlite3

In [57]:
query = """
      CREATE TABLE test
      (a varchar(20),b VARCHAR(20),
      c REAL ,       d INTEGER)
      """
con = sqlite3.connect('mydata.sqlite')
con.execute(query)

In [64]:
con.commit()
data = [('Atlanta','Georgia',1.25,6),
        ('Tallahassee','Florida',2.5,3),
        ('Sacramento','California',1.7,5)]
stmt = "INSERT INTO test VALUES(?,?,?,?)"
con.executemany(stmt,data)
cursor = con.execute('select * from test')
rows = cursor.fetchall()
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.5, 3),
 ('Sacramento', 'California', 1.7, 5),
 ('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.5, 3),
 ('Sacramento', 'California', 1.7, 5),
 ('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.5, 3),
 ('Sacramento', 'California', 1.7, 5),
 ('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.5, 3),
 ('Sacramento', 'California', 1.7, 5)]

In [62]:
cursor.description        #将数据传入到DataFrame构造器 需要知道列名 

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [63]:
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.50,3
2,Sacramento,California,1.70,5
3,Atlanta,Georgia,1.25,6
4,Tallahassee,Florida,2.50,3
5,Sacramento,California,1.70,5
6,Atlanta,Georgia,1.25,6
7,Tallahassee,Florida,2.50,3
8,Sacramento,California,1.70,5


In [66]:
import sqlalchemy as sqla
db = sqla.create_engine('sqlite:///mydata.sqlite')
pd.read_sql('select*from test',db)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.50,3
2,Sacramento,California,1.70,5
3,Atlanta,Georgia,1.25,6
4,Tallahassee,Florida,2.50,3
5,Sacramento,California,1.70,5
6,Atlanta,Georgia,1.25,6
7,Tallahassee,Florida,2.50,3
8,Sacramento,California,1.70,5
